# Random Sequence Generate V2
- sequence의 일부분을 random한 amino-acid로 변환해 sequence 생성
- sequence 변이 부분 중 1개 aa만 변환되도록 생성
    - e.g. 변이 부분 10인 경우 10*19개 seqeucne 생성
    - 각 변이 부분은 현재 aa 제외한 19개 aa로 변이 가능하므로
- csv file로 export
- sequence에 대해 structure prediction 진행

In [1]:
import random
import csv
import os

## sequence 입력
- 변환할 부분 지정

Nb의 경우 n_aa = 100, n_change = 15

In [15]:
sequence = str(input('sequence를 입력하세요: '))
n_aa = int(input('변경 시작할 amino acid 번호를 입력하세요: ')) - 1 # str index 0부터 시작하므로 -1
n_change = int(input('변경할 amino acid 수를 입력하세요: '))
name = input('파일 이름을 입력하세요: ')
csv_filename = f'{name}_replaced_one_aa_sequences.csv'

In [16]:
sequence[n_aa]

'V'

## Random AA로 변환

In [17]:
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

In [18]:
import csv

# Amino acids
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

def mutate_sequence(sequence, n_aa, n_change):
    mutated_sequences = []

    # 변이할 구간
    segment_to_mutate = sequence[n_aa:n_aa + n_change]
    
    for i in range(n_change):
        for aa in amino_acids:
            if segment_to_mutate[i] != aa:  # 원래 아미노산과 다른 경우에만 변이 수행
                mutated_sequence = (
                    sequence[:n_aa + i] + aa + sequence[n_aa + i + 1:]
                )
                mutated_sequences.append(mutated_sequence)

    return mutated_sequences

def save_mutated_sequences_to_csv(sequence, n_aa, n_change, filename='mutated_sequences.csv'):
    mutated_sequences = mutate_sequence(sequence, n_aa, n_change)
    
    # CSV 파일에 변이된 시퀀스 저장
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Mutated Sequence'])
        
        for mutated_sequence in mutated_sequences:
            csvwriter.writerow([mutated_sequence])


In [19]:
# 변이된 시퀀스를 CSV 파일로 저장
save_mutated_sequences_to_csv(sequence, n_aa, n_change, csv_filename)

## Structure Prediction

In [7]:
from ImmuneBuilder import NanoBodyBuilder2

In [20]:
def Run_NanoBodyBuilder2(csv_path):
    name = input('파일 이름을 입력하세요: ')
    output_dir = os.path.join(os.getcwd(), name)
    os.makedirs(output_dir, exist_ok=True)
    try:
        with open(csv_path, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # 헤더를 건너뜁니다

            for idx, row in enumerate(reader):
                if len(row) > 0:
                    sequence = {'H': row[0]}
                    predictor = NanoBodyBuilder2()
                    output_file = os.path.join(output_dir, f'{name}_{idx}.pdb')
                    nanobody = predictor.predict(sequence)
                    nanobody.save(output_file)

    except FileNotFoundError:
        print(f"Error: 파일 '{csv_path}'을 찾을 수 없습니다.")

In [22]:
Run_NanoBodyBuilder2('Nb51_replaced_unique_sequences.csv')